In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
from UltraMotionCapture import kps

vicon = kps.MarkerSet()
vicon.load_from_vicon(
    '../UltraMotionCapture/data/6kmh_softbra_8markers_1.csv',
)
vicon.interp_field()

In [ ]:
from UltraMotionCapture import obj3d

o3_ls = obj3d.load_obj_series(
    folder='../UltraMotionCapture/data/6kmh_softbra_8markers_1/',
    start=0,
    stride=1,
    end=1,
    obj_type=obj3d.Obj3d_Deform
)

In [ ]:
from UltraMotionCapture import obj4d
from probreg import cpd

o4 = obj4d.Obj4d_Deform(
    fps=120,
    enable_rigid=False,
    enable_nonrigid=True,
)
o4.add_obj(*o3_ls, method=cpd.registration_cpd)
o4.load_markerset('M8', vicon)

In [ ]:
import pyvista as pv
pv.set_jupyter_backend('pythreejs')

o3_ls[0].show()

In [ ]:
o3_ls[0].show(mode='deform')

In [ ]:
o3_ls[0].show(mode='diff_deform_gt', obj3d_gt=o3_ls[1])